In [34]:
# Import the required modules and functions

import datetime
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

In [35]:
## Download Files in case of Linux

!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [36]:
## Unzip Files in case of Linux

!unzip datasource.zip -d dealership_data

'unzip' is not recognized as an internal or external command,
operable program or batch file.


About the Data
The file dealership_data contains CSV, JSON, and XML files for used car data which contain features named car_model, year_of_manufacture, price, and fuel.

In [37]:
## Set Paths
tmpfile="dealership_temp.tmp"                   # file used to store all extracted data
logfiles="dealership_logfile.txt"               # all event logs will be stored in this file
targetfile="dealership_transformed_data.csv"    # file where transformed data is stored

In [38]:
### CSV Extract Function
def extract_from_csv(file_to_process):
    dataframe=pd.read_csv(file_to_process)
    return dataframe

In [39]:
### JSON Extract Function
def extract_from_json(file_to_process):
    dataframe=pd.read_json(file_to_process, lines=True)
    return dataframe

In [40]:
#### XML Extract Function
from datetime import date


def extract_from_xml(file_to_process):
    dataframe=pd.DataFrame(columns=["car_model","year_of_manufacture","price","fuel"])
    tree=ET.parse(file_to_process)
    root=tree.getroot()
    for car in root:
        car_model=str(car.find("car_model").text)
        year_of_manufacture=int(car.find("year_of_manufacture").text)
        price=float(car.find("price").text)
        fuel=str(car.find("fuel").text)
        dataframe=dataframe.append({"car_model":car_model,"year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe

In [41]:
### Extract Function

def extract():
    extracted_data1=pd.DataFrame(columns=['car_model','year_of_manufacture','price','fuel'])

    #process all csv files
    for csvfile in glob.glob("D:\IBM Data Engineering\Course 3 Python Project for Data Engineering\dealership_data\*.csv"):
        extracted_data1=extracted_data1.append(extract_from_csv(csvfile), ignore_index=True)

    #process all json files
    for jsonfile in glob.glob("D:\IBM Data Engineering\Course 3 Python Project for Data Engineering\dealership_data\*.json"):
        extracted_data1=extracted_data1.append(extract_from_json(jsonfile), ignore_index=True)

    #process all xml files
    for xmlfile in glob.glob("D:\IBM Data Engineering\Course 3 Python Project for Data Engineering\dealership_data\*.xml"):
        extracted_data1=extracted_data1.append(extract_from_xml(xmlfile), ignore_index=True)

    return extracted_data1

Transform
The transform function does the following tasks.

1.  Convert height which is in inches to millimeter
2.  Convert weight which is in pounds to kilograms

In [44]:
def transform(data):
    data["price"]=round(data.price,2)
    return data

Loading

In [45]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)

Logging

In [46]:
def log(message):
    timestamp_format='%Y-%h-%d-%H:%M:%S'
    now=datetime.now()
    timestamp=now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message +'\n')

Running ETL Process

In [47]:
log("ETL Job Started")

In [48]:
log("Extract Phase Started")
extracted_data1=extract()
log("Extract Phase Ended")
extracted_data1

C:\Users\deepa\AppData\Local\Temp\ipykernel_18936\372733501.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data1=extracted_data1.append(extract_from_csv(csvfile), ignore_index=True)
C:\Users\deepa\AppData\Local\Temp\ipykernel_18936\372733501.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data1=extracted_data1.append(extract_from_csv(csvfile), ignore_index=True)
C:\Users\deepa\AppData\Local\Temp\ipykernel_18936\372733501.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data1=extracted_data1.append(extract_from_csv(csvfile), ignore_index=True)
C:\Users\deepa\AppData\Local\Temp\ipykernel_18936\372733501.py:12: FutureWarning: The frame.append method is deprecated and will be removed fr

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.000000,Petrol
1,sx4,2013,7089.552239,Diesel
2,ciaz,2017,10820.895522,Petrol
3,wagon r,2011,4253.731343,Petrol
4,swift,2014,6865.671642,Diesel
...,...,...,...,...
85,camry,2006,3731.343284,Petrol
86,land cruiser,2010,52238.805970,Diesel
87,corolla altis,2012,8805.970149,Petrol
88,etios liva,2013,5149.253731,Petrol


In [49]:
log("Transform Phase Started")
transformed_data=transform(extracted_data1)
log("Transform Phase Ended")
transformed_data

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel
...,...,...,...,...
85,camry,2006,3731.34,Petrol
86,land cruiser,2010,52238.81,Diesel
87,corolla altis,2012,8805.97,Petrol
88,etios liva,2013,5149.25,Petrol


In [50]:
log("Load Phase Started")
load(targetfile, transformed_data)
log("Load Phase Ended")

In [51]:
log("ETL Job Ended")

In [54]:
f = open("transformed_data.csv", "r")
print(f.read())

,name,height,weight
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45
5,alex,1.67,51.25
6,ajay,1.82,61.91
7,alice,1.76,69.41
8,ravi,1.73,64.56
9,joe,1.72,65.45
10,alex,1.67,51.25
11,ajay,1.82,61.91
12,alice,1.76,69.41
13,ravi,1.73,64.56
14,joe,1.72,65.45
15,jack,1.74,55.93
16,tom,1.77,64.18
17,tracy,1.78,61.9
18,john,1.72,50.97
19,jack,1.74,55.93
20,tom,1.77,64.18
21,tracy,1.78,61.9
22,john,1.72,50.97
23,jack,1.74,55.93
24,tom,1.77,64.18
25,tracy,1.78,61.9
26,john,1.72,50.97
27,simon,1.72,50.97
28,jacob,1.7,54.73
29,cindy,1.69,57.81
30,ivan,1.72,51.77
31,simon,1.72,50.97
32,jacob,1.7,54.73
33,cindy,1.69,57.81
34,ivan,1.72,51.77
35,simon,1.72,50.97
36,jacob,1.7,54.73
37,cindy,1.69,57.81
38,ivan,1.72,51.77

